In [ ]:
#|default_exp deck

In [ ]:
#|hide
from nbdev.showdoc import *

# Deck
> Playing Cards

In [ ]:
#|export
from python_swak.card import *
from fastcore.basics import *
from fastcore.test import *
import random

In [ ]:
#|export
class Deck:
    "Represents a deck of cards"
    def __init__(self): self.cards = [Card(s, r) for s in range(4) for r in range(1, 14)]
    def __str__(self): return '; '.join(map(str, self.cards))
    def __len__(self): return len(self.cards)
    def __contains__(self, card): return card in self.cards
    __repr__ = __str__
    
    def add(self,
            card:Card): # Card to add
        "Adds `card` to the deck"
        self.cards.append(card)

    def remove(self,
               card:Card): # Card to remove
        "Removes `card` from the deck or raises exception if it is not there"
        self.cards.remove(card)

    def shuffle(self):
        "Shuffles the cards in this deck"
        random.shuffle(self.cards)

A Deck of cards is a collection of `Card` objects:

In [ ]:
deck = Deck()
deck

A♣️; 2♣️; 3♣️; 4♣️; 5♣️; 6♣️; 7♣️; 8♣️; 9♣️; 10♣️; J♣️; Q♣️; K♣️; A♦️; 2♦️; 3♦️; 4♦️; 5♦️; 6♦️; 7♦️; 8♦️; 9♦️; 10♦️; J♦️; Q♦️; K♦️; A❤️; 2❤️; 3❤️; 4❤️; 5❤️; 6❤️; 7❤️; 8❤️; 9❤️; 10❤️; J❤️; Q❤️; K❤️; A♠️; 2♠️; 3♠️; 4♠️; 5♠️; 6♠️; 7♠️; 8♠️; 9♠️; 10♠️; J♠️; Q♠️; K♠️

There are 52 cards in a deck.

In [ ]:
test_eq(len(deck), 52)

In [ ]:
#|export
@patch
def pop(self:Deck,
        index=-1): # Card number to pop
    "Removes and returns card `index` from the deck"
    return self.cards.pop(index)

In [ ]:
deck.pop()

K♠️

There are 51 cards left in the deck now.

In [ ]:
test_eq(len(deck), 51)

You can show the docs for methods not created with `patch` by calling `show_doc`. For example, the code `show_doc(Deck.remove)` produces the following documentation:

In [ ]:
show_doc(Deck.remove)

---

[source](https://github.com/shadisharba/python_swak/blob/main/python_swak/deck.py#L28){target="_blank" style="float:right; font-size:smaller"}

### Deck.remove

>      Deck.remove (card:python_swak.card.Card)

*Removes `card` from the deck or raises exception if it is not there*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| card | Card | Card to remove |

If we remove a card from the Deck we can verify that it no longer exists:

In [ ]:
card23 = Card(2, 3)
deck.remove(card23)

assert card23 not in deck



However, another card that we haven't removed, such as the `10 of hearts` will still be in the Deck of cards because we haven't removed it:


In [ ]:
assert Card(2,10) in deck

In [ ]:
#|export
class Hand(Deck):
    def __init__(self):
        super().__init__()
        self.cards.clear()

In [ ]:
hand = Hand()
test_eq(len(hand), 0)

In [ ]:
#|export
def move_cards(source:Deck, # deck to move cards from
               dest:Hand, # destination to move cards to
               num:int): # number of cards to move
    "Pop the given number of cards from the deck and move to `dest`."
    for i in range(num): dest.add(source.pop())

:::{.callout-note}

You might be wondering: "what are these comments are following each parameter?"  These are called [docments](https://fastcore.fast.ai/docments.html), a concise way of documenting your code that also renders beautifully in nbdev.  nbdev also supports rendering [numpy-style docstrings](https://numpydoc.readthedocs.io/en/latest/format.html) as well.

:::

In [ ]:
deck = Deck()
deck.shuffle()

In [ ]:
move_cards(deck, hand, 7)
hand

2♠️; 6❤️; 10♠️; 2❤️; 3♣️; 2♦️; 2♣️

In [ ]:
test_eq(len(deck), 52-7)
test_eq(len(hand), 7)

## Drawing Cards With Replacement

Let's try something fun with our deck of cards, drawing a card with replacement:

In [ ]:
#|export
def draw_n(n:int, # number of cards to draw
           replace:bool=True): # whether or not draw with replacement
    "Draw `n` cards, with replacement iif `replace`"
    d = Deck()
    d.shuffle()
    if replace: return [d.cards[random.choice(range(len(d.cards)))] for _ in range(n)]
    else: return d.cards[:n]

In [ ]:
sample = draw_n(10)
sample

[Q♣️, J♠️, 6♣️, 7♦️, 3♠️, 7♦️, 7♣️, 5♠️, 4♠️, 6♣️]

In [ ]:
#|hide
assert len(sample) == 10

## Visualizing the results

This isn't terribly interesting from a statistical perspective.  However, its an example of how you can include visualizations in your nbdev projects!  

:::{.callout-note}

Notice how we are hiding just the input with `#|echo: false`, so readers can see the output but hide the code.  You can also fold the code using the [`#|code-fold: true`](https://quarto.org/docs/output-formats/html-code.html#folding-code) directive.

:::

In [ ]:
#|echo: false
#|eval: false

import pandas as pd
import plotly.express as px

sampledf = pd.DataFrame([{'suit': c.suit_nm, 'rank': c.rank_nm } for c in draw_n(5000)])
fig = px.bar(sampledf.groupby('suit').count().reset_index().rename(columns={'rank':'count'}),
             x='suit', y='count',  title="Count By Suit On 5000 Random Draws With Replacement")
fig.show()

## Create a CLI (Advanced)

We can create a CLI with `@call parse`

In [ ]:
#|export
from pathlib import Path
from fastcore.script import call_parse

In [ ]:
#|export
@call_parse
def draw_cards(n:int, # number of cards to draw
               replace:bool=True, # whether or not draw with replacement
               outfile:str=None): # output file, defaults to stdout
    "Draw `n` cards optionally with replacement"
    cards = draw_n(n, replace=replace)
    strcards = '\n'.join(map(str, cards))
    print(strcards) if outfile is None else Path(outfile).write_text(strcards, encoding="utf8")

:::{.callout-tip}

We normally wouldn't repeat all of these arguments when one function is wrapping another one.  Instead, we would use [delegates](https://fastcore.fast.ai/meta.html#delegates).  However, we wanted to keep this tutorial simple, so we didn't use that here. 

:::

In [ ]:
fname = 'sample.txt'
draw_cards(10, outfile=None)
# print(Path(fname).read_text(encoding="utf8"))

4❤️
A❤️
4❤️
5❤️
Q♠️
10♦️
9❤️
4♣️
K❤️
J♣️
